# stock price prediction
![](https://miro.medium.com/max/2560/0*R5pC0bAlYxH_nTlF.jpg)

👍**このnotebookが役に立った方は是非いいねをお願いします**👍  
👍**please upvote**👍  
this notebook is created from https://www.kaggle.com/takahiro1127/topix-prediction-of-topix-stock-price  
このnotebookはこちらのnotebook https://www.kaggle.com/takahiro1127/topix-prediction-of-topix-stock-price の発展系として作成しました。

In [ ]:
import os
import numpy as np
import pandas as pd
import xgboost as xgb
import collections
import seaborn as sns
from xgboost import DMatrix
import matplotlib.pyplot as plt
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, recall_score
from sklearn.metrics import cohen_kappa_score
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from scipy import signal
import matplotlib.pyplot as plt
import random
import numpy as np
from statistics import mean, mode
# チャート用設定
# setting for chart
import plotly as py
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=DeprecationWarning)

init_notebook_mode(connected=True)
layout=go.Layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(250,250,250,0.8)')
fig = go.Figure(layout=layout)
templated_fig = pio.to_templated(fig)
pio.templates['my_template'] = templated_fig.layout.template
pio.templates.default = 'my_template'

## EDA / データ探索
data visualization and data analysis is here https://www.kaggle.com/takahiro1127/topix-prediction-of-topix-stock-price  
データ探索と可視化はこちらのnote bookにあります。　https://www.kaggle.com/takahiro1127/topix-prediction-of-topix-stock-price 


## 方針
過去250日間のデータを用いてxgbを学習し251日目の株価の変化を予測する。  
→これを予測したい日数分繰り返す
以下同様に  
```
predict = []
for (int i = first_predict_day; i < final_day; i++) {
  model.train(data[(i - 251) ~ i - 1])
  predict[i] = model.predict(data[i])
}
```
のような形で行う

## plan
train XGB by recent (i - 250)~ i days data and predict (i+1)-day Change rate  
→train XGB by recent (i - 251 + 1)~ (i + 1) days data and predict (i + 1 + 1)-day Change rate  
→predict all day as above
```
predict = []
for (int i = first_predict_day; i < final_day; i++) {
  model.train(data[(i - 251) ~ i - 1])
  predict[i] = model.predict(data[i])
}
```

## 初期設定/setting
直近150日間の予測を行うことにする。  
predict 150days

In [ ]:
df = pd.read_csv("/kaggle/input/topix4/TOPIX4.csv", names=("Date", "Open", "High", "Low", "Close")).drop(0)
prediction_day_count = 150
train_day_count = 250
df = df.loc[len(df) - prediction_day_count - train_day_count - 34:].reset_index().drop('index', 1)

## その他のインデックスファンドのデータを取得する/prepare other index-fund
topix以外のdataも予測に用いることにする。  
コメントアウトをするか、しないかによって用いるかどうかを決定する。  
use other index-fund data  
switch use or unuse by comment-out

In [ ]:
# df = df.loc[len(df) - prediction_day_count - train_day_count - 34 -1:].reset_index().drop('index', 1)
# index_funds = ['N225', 'GSPC', 'EZU']
index_funds = []
# columns_added_by_index_funds = ['N225_Close', 'GSPC_Close', 'EZU_Close']
# columns_added_by_index_funds = ['N225_Close', 'GSPC_Close', 'EZU_Close', 'N225_RSI', 'GSPC_RSI', 'EZU_RSI']
columns_added_by_index_funds = []
# for index_fund in index_funds:
#     index_df = pd.read_csv("/kaggle/input/stock-price-datas/" + index_fund + ".csv", names=("Date", "Open", "High", "Low", index_fund+"_Close", "Adj Close", "Volume")).drop(0)
#     index_df = index_df.drop(["Open", "High", "Low", "Adj Close", "Volume"], 1)
#     index_df["Date"] = index_df["Date"].str.replace('-', '/')
#     df = pd.merge(df, index_df, on="Date", how='left')
#     if (index_fund != 'N225'):
#         df[index_fund+"_Close"] = df[index_fund+"_Close"].shift()
# df = df[1:]      # Because of shifting close price
# df = df.reset_index().drop("index", 1)

## ready features

### フラグの作成 / make flag
ゴールデンクロスの簡易版です  
simple version of golden cross  

In [ ]:
def flag_features(df):
    for num in ['5', '10', '15', '30']:
        # 移動平均線と移動標準偏差
        # moving average and std of it
        df['SMA_' + num] = df['Close'].rolling(int(num)).mean().shift() / df['Close']
        df['SMA_' + num + '_std'] = df['Close'].rolling(int(num)).std().shift() / df['Close']
        col_name = 'SMA_' + num + '_sub'
        today_col_name = 'flag_today_sma_' + num
        yesterday_col_name = 'flag_yesterday_sma_' + num
        df[col_name] = df['Close'].rolling(int(num)).mean().shift()
        df.loc[df[col_name] < df['Close'], today_col_name] = 1
        df.loc[df[col_name] >= df['Close'], today_col_name] = 0
        df[yesterday_col_name] = df[today_col_name].shift(1)
        # フラグを作成
        # make flag
        df.loc[(df[yesterday_col_name] == 0) & (df[today_col_name] == 1), "flag_sma_under_" + num] = 1
        df.loc[~((df[yesterday_col_name] == 0) & (df[today_col_name] == 1)), "flag_sma_under_" + num] = 0
        df.loc[(df[yesterday_col_name] == 1) & (df[today_col_name] == 0), "flag_sma_over_" + num] = 1
        df.loc[~((df[yesterday_col_name] == 1) & (df[today_col_name] == 0)), "flag_sma_over_" + num] = 0
        df = df.drop([col_name, yesterday_col_name, today_col_name], 1)
    up_flag = {'5' : 0, '10': 0, '15' : 0, '30' : 0}
    down_flag = {'5' : 0, '10': 0, '15' : 0, '30' : 0}
    # フラグから特定の距離にあるかチェック
    # check distance from flag
    for i in range(len(df)):
        for num in ['5', '10', '15', '30']:
            up_column = "up_flag_distance_" + num
            down_column = "down_flag_distance_" + num
            df.loc[i, up_column] = 0
            df.loc[i, down_column] = 0
            if up_flag[num] > int(num):
                up_flag[num] = 0
                df.loc[i, "up_flag_distance_" + num] = 0
            elif up_flag[num] > 0:
                df.loc[i, "up_flag_distance_" + num] = up_flag[num]
                up_flag[num] += 1

            if down_flag[num] > int(num):
                down_flag[num] = 0
                df.loc[i, "down_flag_distance_" + num] = 0
            elif down_flag[num] > 0:
                df.loc[i, "down_flag_distance_" + num] = down_flag[num]
                down_flag[num] += 1

            if df.loc[i, "flag_sma_under_" + num] == 1:
                df.loc[i, "down_flag_distance_" + num] = 1
                down_flag[num] = 2
            if df.loc[i, "flag_sma_over_" + num] == 1:
                df.loc[i, "up_flag_distance_" + num] = 1
                up_flag[num] = 2
    for num in ['5', '10', '15', '30']:
        df = df.drop(["flag_sma_over_" + num, "flag_sma_under_" + num], 1)
    return df

### RSI / 相対力指数
相対的に上昇方向なのか下降方向なのかを示す指数です。  
relative strength of up or down

In [ ]:
def relative_strength_idx(df, close_column = "Close", n=14):
    close = df[close_column]
    delta = close.diff()
    delta = delta[1:]
    pricesUp = delta.copy()
    pricesDown = delta.copy()
    pricesUp[pricesUp < 0] = 0
    pricesDown[pricesDown > 0] = 0
    rollUp = pricesUp.rolling(n).mean()
    rollDown = pricesDown.abs().rolling(n).mean()
    rs = rollUp / rollDown
    rsi = 100.0 - (100.0 / (1.0 + rs))
    return rsi

### wavelet analysis / ウェーブレット解析
Wavelet coefficient represents periodic nature of time-series data.  
ウェーブレット係数は時系列データの周期的な性質を表しています。

In [ ]:
def wt( data ):
    widths = np.arange(1, 100)
    return signal.cwt( data, signal.ricker, widths )

### make objective variable / 目的変数に作成
make objective variable and assign weight based on objective variable(change label)  
目的関数を作成します。また、変化率に基づいて学習の重み付けを行います。

#### weight
big change rate gives me big profit(or loss).  
we must focus on big change.  
set heavy weight on big change.  

大きな変化は大きな利益や損失をもたらすため、間違えた時のリスクが大きいです。  
よって、大きな変化ほど予測性能があがるように重み付けをします。

In [ ]:
def change_rate(df):
    # 評価に終値の変化率を使うため
    # qualtile([0.2, 0.4, 0.6, 0.8])で閾値を決定
    # 大きなプラスを2として、小さなプラスを1 →　大きなマイナスを-2
    
    # objective variable is change rate of close
    # devide change rate to label by quantile
    # big plus is 2 and small plus is 1 ... bug minus is -2
    df['Change_rate'] = df['Close'].pct_change()
    divide = df['Change_rate'].quantile([0, 0.2, 0.4, 0.6, 0.8, 1.0])
    df.loc[(df['Change_rate'] > divide[0.4]) & (df['Change_rate'] < divide[0.6]), 'Change_label'] = 0
    df.loc[(df['Change_rate'] > divide[0.4]) & (df['Change_rate'] < divide[0.6]), 'weight'] = 1
    df.loc[(df['Change_rate'] >= divide[0.2]) & (df['Change_rate'] <= divide[0.4]), 'Change_label'] = -1
    df.loc[(df['Change_rate'] >= divide[0.2]) & (df['Change_rate'] <= divide[0.4]), 'weight'] = 1.2
    df.loc[df['Change_rate'] < divide[0.2], 'Change_label'] = -2
    df.loc[df['Change_rate'] < divide[0.2], 'weight'] = 1.5
    df.loc[(df['Change_rate'] <= divide[0.8]) & (df['Change_rate'] >= divide[0.6]), 'Change_label'] = 1
    df.loc[(df['Change_rate'] <= divide[0.8]) & (df['Change_rate'] >= divide[0.6]), 'weight'] = 1.2
    df.loc[df['Change_rate'] > divide[0.8], 'Change_label'] = 2
    df.loc[df['Change_rate'] > divide[0.8], 'weight'] = 1.5
    df = df.reset_index()
    return df.drop('index', 1)

### ready simple features / 単純な説明変数の作成
MACD and EMA... is very easy to create.
In this method we make easy features.

簡単に作成できる特徴量はここで作成しています。

In [ ]:
def ready_features(df):
    # データの読み込み
    # read csv and modify type of columns
    df['Date'] = pd.to_datetime(df['Date'])
    df["Open"] = df['Open'].astype(float)
    df["Close"] = df['Close'].astype(float)
    df["High"] = df['High'].astype(float)
    df["Low"] = df['Low'].astype(float)
    for index_fund in index_funds:
        df[index_fund+"_Close"] = df[index_fund+"_Close"].astype(float)
    # 指数平滑移動平均
    # Exponential Moving Average
    # 指数平滑移動平均の日本語の説明↓
    # https://media-kojirokousi.com/exponential-moving-average/#:~:text=%E6%8C%87%E6%95%B0%E5%B9%B3%E6%BB%91%E7%A7%BB%E5%8B%95%E5%B9%B3%E5%9D%87%E7%B7%9A(EMA)%E3%81%AF%E3%80%81%E5%BE%93%E6%9D%A5%E3%81%AE,EMA%E3%81%A8%E5%91%BC%E3%81%B0%E3%82%8C%E3%81%BE%E3%81%99%E3%80%82
    df['EMA_9'] = df['Close'].ewm(9).mean().shift() / df['Close']
    df['EMA_9_std'] = df['Close'].ewm(9).std().shift() / df['Close']
    # MACD
    # MACDの日本語の説明↓
    # https://www.sevendata.co.jp/shihyou/technical/macd.html
    EMA_12 = pd.Series(df['Close'].ewm(span=12, min_periods=12).mean())
    EMA_26 = pd.Series(df['Close'].ewm(span=26, min_periods=26).mean())
    df['MACD'] = pd.Series(EMA_12 - EMA_26)/ df['Close']
    df['MACD_signal'] = pd.Series(df.MACD.ewm(span=9, min_periods=9).mean())/ df['Close']
    df['RSI'] = relative_strength_idx(df).fillna(0)
    df = flag_features(df)
    df = change_rate(df)
    #↓他のインデックスファンドのデータも同様の特徴量を生成している
    #↓create same features from other index-fund data
    for index_fund in index_funds:
        close_column = index_fund+"_Close"
        rsi_column = index_fund +"_RSI"
        df[index_fund + '_EMA_9'] = df[close_column].ewm(9).mean().shift() / df[close_column]
        df[index_fund + '_EMA_9_std'] = df[close_column].ewm(9).std().shift() / df[close_column]
        EMA_12 = pd.Series(df[close_column].ewm(span=12, min_periods=12).mean())
        EMA_26 = pd.Series(df[close_column].ewm(span=26, min_periods=26).mean())
        df[rsi_column] = relative_strength_idx(df, close_column).fillna(0)
        column_wave = wt(df[close_column])
        for num in ['5', '10', '15', '30']:
            df[close_column + '_wave_' + num] = column_wave[int(num)]
        rsi_wave = wt(df[rsi_column])
        for num in ['5', '10', '15', '30']:
            df[rsi_column + '_wave_' + num] = rsi_wave[int(num)]

    # wavelet解析
    # wavelet
    for column in ['Close', 'RSI']:
        column_wave = wt(df[column])
        for num in ['5', '10', '15', '30', '45', '50', '75','98']:
            df[column + '_wave_' + num] = column_wave[int(num)]
    return df


df = ready_features(df)
df.tail()

### ready data for train / train用にデータを作成

In [ ]:
def ready_data_for_train(df_for_ready, day_count): 
    df_for_ready = modify_weight_based_on_day_count(df_for_ready, day_count)
    df_for_ready, train_df, valid_df, test_df = modify_and_split_data_for_xgb(df_for_ready, day_count)
    X_train, y_train = get_label_and_explanatory_variable(train_df)
    X_valid, y_valid = get_label_and_explanatory_variable(valid_df)
    X_test, y_test = get_label_and_explanatory_variable(test_df)
    eval_set = [(X_train, y_train), (X_valid, y_valid)]
    weights = X_train["weight"].astype('float')
    X_train = X_train.drop("weight", axis=1)
    X_valid = X_valid.drop("weight", axis=1)
    X_test = X_test.drop("weight", axis=1)
    eval_set = [(X_train, y_train), (X_valid, y_valid)]
    return eval_set, X_train, y_train, X_valid, y_valid, X_test, y_test, weights, df_for_ready, test_df
    
def modify_weight_based_on_day_count(df_for_ready, day_count):
    df_for_ready = df_for_ready.reset_index()
    df_for_ready.loc[df_for_ready['weight'] - 0.5 > df_for_ready['weight'] - 0.7 + 1.2 * df_for_ready['index']/len(df_for_ready), 'weight'] = df_for_ready['weight'] - 0.5
    df_for_ready.loc[df_for_ready['weight'] - 0.5 <= df_for_ready['weight'] - 0.7 + 1.2 * df_for_ready['index']/len(df), 'weight'] = df_for_ready['weight'] - 0.7 + 1.2 * df_for_ready['index']/len(df_for_ready)
    return df_for_ready.drop('index', 1)
    
def modify_and_split_data_for_xgb(df_for_ready, day_count):
    # 前日のデータから、翌日の終値を予測するため、closeをshiftしている。
    # ↓we would like to predict tommorow's movement
    df_for_ready['Change_label'] = df_for_ready['Change_label'].shift(-1)
    df_for_ready['Change_rate'] = df_for_ready['Change_rate'].shift(-1)
    df_for_ready['weight'] = df_for_ready['weight'].shift(-1)

    df_for_ready = df_for_ready.iloc[33:] # Because of moving averages and MACD line
    df_for_ready = df_for_ready[:-1]      # Because of shifting close price
    df_for_ready["Change_label"] = df_for_ready["Change_label"].astype('int')
    df_for_ready = df_for_ready.reset_index()
    df_for_ready = df_for_ready.drop(['index'], 1)
    test_df = df_for_ready.loc[len(df_for_ready) - prediction_day_count + day_count: len(df_for_ready) - prediction_day_count + day_count].copy()
    train_df = df_for_ready.loc[: len(df_for_ready) - prediction_day_count + day_count - 1].copy()
    train_df, valid_df = train_test_split(train_df, test_size=0.19, stratify=train_df.Change_label)
    return df_for_ready,train_df, valid_df, test_df
def get_label_and_explanatory_variable(df_for_ready):
    drop_cols = ['Date', 'Open', 'Low', 'High', 'Close', 'Change_rate']
    df_for_ready = df_for_ready.drop(drop_cols, 1)
    df_for_ready = df_for_ready.drop(columns_added_by_index_funds, 1)
    label = df_for_ready['Change_label'].copy()
    explanatory = df_for_ready.drop(columns='Change_label')
    return explanatory, label

## 学習と予測/train and prediction
xgbを用いて学習する。  
また、初期状態に依存してしまうことから、アンサンブル学習を行うことにする。  
predict by xgb  
prediction is influenced by initial state, prevent influence by using ensemble learning

In [ ]:
result = df[df.Date == "aa"].copy().reset_index().drop(['index'], 1)
#↑データのフレームのみをコピー
#↑copy frame only
ensemble_count = 10
for i in tqdm(range(prediction_day_count)):
    predicts = []
    for j in range(ensemble_count):
        eval_set, X_train, y_train, X_valid, y_valid, X_test, y_test, weights, ans_df, test_df = ready_data_for_train(df.loc[i:].copy(), i)
        model = xgb.XGBClassifier(n_estimators=200, 
            learning_rate=0.05,
            max_depth= 8,
            gamma=0.02,
            random_state= 42,
            num_class =5,
            objective='multi:softprob')
        model.fit(X_train, y_train, eval_set=eval_set, verbose=False, sample_weight = weights)
        predicts.append(model.predict(X_test)[0])
        if ensemble_count - 1 == j:
            test_df['pred'] = collections.Counter(predicts).most_common()[0][0]
    result = pd.concat([result, test_df.reset_index()])
result = result.reset_index().drop(['index', 'level_0'], 1)
acc = accuracy_score(result['Change_label'], result['pred'])
print(acc)

## シミュレーション / simulation
損益シミュレーションを以下の形で行う  
simulate profit and loss as below  

W_0 = 1

W_{t+1} =  ( 1 + r_{t+1} * w_t ) W_t

### 記号

* W_t : 時刻tでの保有資産額 / money at t-time  
* r_t : 時刻{t-1} から 時刻tまでのTOPIX変化率 / change rate from (t-1)-time to t-time  
* w_t : 自分の投資比率 / investment rate  

### 予測と投資比率の関係 / decide investment rate by prediction
* +2 : 大幅なTOPIX上昇/big plus  => w_t =  1.0 
* +1 : 小幅なTOPIX上昇/small plus  => w_t =  0.5
*  0 : TOPIX変化なし/no channge  　=> w_t =  0.0
* -1 : 小幅なTOPIX下落/small minus  => w_t = -0.5
* -2 : 大幅なTOPIX下落/big minus  => w_t = -1.0

In [ ]:
def simulate_based_on_predict(sim_data):
    money_his = pd.DataFrame(columns = ['Date', 'money'])
    money = 1
    for i in range(0, len(result)):
        data = sim_data.loc[i].copy()
        today_pred = data.pred
        Change_rate = data.Change_rate
        if (today_pred == -2):
            money = (1 + Change_rate * -1) * money
        elif (today_pred == -1):
            money = (1 + Change_rate * -0.5) * money
        elif (today_pred == 1):
            money = (1 + Change_rate * 0.5) * money
        elif (today_pred == 2):
            money = (1 + Change_rate * 1) * money
        money_his.loc[data.Date] = [data.Date, money]
    return money, money_his
money, money_his = simulate_based_on_predict(result.copy())
print(money)
fig = go.Figure(go.Scatter(x=money_his.Date, y=money_his.money, name='money_history'))
fig.show()

In [ ]:
print(confusion_matrix(result['Change_label'], result['pred']))
sns.heatmap(confusion_matrix(result['Change_label'], result['pred']))

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10, 10))
plot_importance(booster=model, ax=ax)

In [ ]:
fig = make_subplots(rows=1, cols=1)
fig.add_trace(go.Scatter(x=result.Date, y=result.Change_label,
                         name='Truth'), row=1, col=1)

fig.add_trace(go.Scatter(x=result.Date,
                         y=result.pred,
                         name='Prediction'), row=1, col=1)

fig.show()